# google drive 연결

In [1]:
import os

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!rm -rf "/content/openpose"

# cmake & git clone & Openpose build copy 
 - google drive : My Drive/tmp/openpose  
 - 원본 : !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc` -> 결과 google drive로 cp
 - copy 후 openpose.bin 실행권한 부여

In [3]:
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz

In [4]:
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
# clone openpose
!git clone -q --depth 1 $git_repo_url
!sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
# install system dependencies
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev

Selecting previously unselected package libgflags2.2.
(Reading database ... 144487 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

In [5]:
!cp -r "/content/drive/My Drive/tmp/openpose/" /content/

In [ ]:
!ls -ltr /content/openpose/build/examples/openpose
!chmod u+x /content/openpose/build/examples/openpose/openpose.bin
!ls -ltr /content/openpose/build/examples/openpose

# Input data copy

In [6]:
!rm -rf "./yoga_video/"
os.mkdir("./yoga_video/")

!wget -q https://archive.org/download/YogaVidCollected/Yoga_Vid_Collected.zip 

In [ ]:
!unzip Yoga_Vid_Collected.zip -d yoga_video

# Openpose model/input/output path

In [ ]:
import glob
import os
from google.colab import files

model_path = '/content/openpose'  
input_path = '/content/yoga_video'
output_path = '/content/output'
output_file_path = "".join([output_path, '/file'])
output_json_path = "".join([output_path, '/json'])

os.system(f"rm -rf '{output_path}'")
os.mkdir(output_path)
os.mkdir(output_file_path)
os.mkdir(output_json_path)

input_file_type = "mp4"
input_cut_time = 10

%cd $model_path

# Pose Detection

In [ ]:
def detect_keypoint_file(file_name):

  origin_file_path = "".join([output_path, '/', file_name.split('/')[-1]])
  origin_file_name = file_name.split('/')[-1].split('.')[0]

  temp_file_name = "".join([output_path, '/', 'openpose.avi'])
  output_file_name = "".join([output_file_path, '/', origin_file_name, '.', input_file_type])
  output_json_name = "".join([output_json_path, '/', origin_file_name])
    
  os.system(f"ffmpeg -y -loglevel info -i '{file_name}' -t {input_cut_time} '{origin_file_path}'")
  os.system(f"./build/examples/openpose/openpose.bin --keypoint_scale 3 --frame_step 3 --video '{origin_file_path}' --write_json '{output_json_name}/' --display 0 --write_video '{temp_file_name}'")
  os.system(f"ffmpeg -y -loglevel info -i '{temp_file_name}' '{output_file_name}'")
  os.system(f"rm -rf '{origin_file_path}'")
  os.system(f"rm -rf '{temp_file_name}'")
  #!ffmpeg -y -loglevel info -i '/content/yoga_video/Abhay_Bhuj.mp4' -t 10 '/content/output/Abhay_Bhuj.mp4'
  #!./build/examples/openpose/openpose.bin --keypoint_scale 3 --frame_step 3 --video '/content/output/Abhay_Bhuj.mp4' --write_json '/content/output/json/Abhay_Bhuj/' --display 0 --write_video '/content/output/openpose.avi'

# Download to local

In [ ]:
def download_file():
  file_zip_name = "".join([output_file_path, '.zip'])
  json_zip_name = "".join([output_json_path, '.zip'])

  os.system(f"zip -s 25 '{file_zip_name}' '{output_file_path}'")
  files.download(file_zip_name)
  
  os.system(f"zip -s 25 '{json_zip_name}' '{output_json_path}'")
  files.download(json_zip_name)

# Main

In [ ]:
for file_name in glob.glob("".join([input_path, '/*', input_file_type])):   
  detect_keypoint_file(file_name)

# compute_OKS_rescale / compute_PDJ_rescale

In [ ]:
from os import listdir
from os.path import isfile, join
import json
from scipy.spatial import distance
import math

# json파일 폴더에서 180개의 json파일을 읽어 (x, y) tuple이 들어있는 4500 (25 keypoints * 180 frame) 벡터 추출
def read_vec(json_path):
    json_path = json_path
    json_files = [f for f in listdir(json_path) if isfile(join(json_path, f))]
    json_files.sort()

    keypoint_data = []
    for json_file in json_files:
        #print(json_file)
        with open(json_path+json_file) as f:
            data = json.load(f)
        ary = data["people"][0]["pose_keypoints_2d"]
        del ary[2::3]

        # 코의 좌표 추출
        nose_x = ary[0]
        nose_y = ary[1]

        odd = ary[0::2]
        even = ary[1::2]

        # 코의 좌표가 (0, 0)이 되게끔 모든 keypoint 위치를 이동 (shift)
        origin_x = [x - nose_x for x in odd]
        origin_y = [y - nose_y for y in even]

        # 각 keypoint의 위치 정보를 (x, y)로 묶어서 list 안에 넣음
        keypoint_data += tuple(zip(origin_x, origin_y))

    return keypoint_data

# PDJ (Percentage of Detected Joints) 계산식 중 diagonal에 해당하는 값을 base (length between nose and middle hip)의 형태로 구함
def get_base(v1, v2):
    # 스승과 학생의 base 길이를 각각 구함
    base1 = distance.euclidean(v1[0], v1[8]) # 스승
    base2 = distance.euclidean(v2[0], v2[8]) # 학생

    scaling_factor = base1 / base2
    
    # 두 base의 평균을 기준 길이(PDJ의 diagonal)로 정함
    result = (base1 + base2) / 2
    return scaling_factor, result


# OKS (Object Keypoint Similarity)를 계산
# OKS = exp(-1.0 * (di ** 2) / (2 * alpha * base ** 2))
def compute_oks(v1, v2, alpha):
    scaling_factor, base = get_base(v1, v2)
    nose_to_hip_len = alpha * base

    oks = 0
    for i in range(len(v1)):
        # 학생(v2)의 좌표값에 대해 scaling_factor를 곱해 스승에 맞춤 
        v2_scaled = tuple(scaling_factor * val for val in v2[i])

        pointwise_dist = distance.euclidean(v1[i], v2_scaled)
        oks += math.exp(-1.0 * pointwise_dist ** 2 / (2 * nose_to_hip_len ** 2))

    result = oks / len(v1)
    return result

# PDJ (Percentage of Detected Joints)를 계산
def compute_pdj(v1, v2, alpha):
    scaling_factor, base = get_base(v1, v2)
    nose_to_hip_len = alpha * base

    pdj = 0
    for i in range(len(v1)):
        # 학생(v2)의 좌표값에 대해 scaling_factor를 곱해 스승에 맞춤 
        v2_scaled = tuple(scaling_factor * val for val in v2[i])
        #print(v2_scaled)
        pointwise_dist = distance.euclidean(v1[i], v2_scaled)
        if pointwise_dist < nose_to_hip_len:
             pdj += 1

    result = pdj * 1.0 / len(v1)
    return result

In [ ]:
output_path_1 = '/content/output/json/Bhumi_Padam/'
output_path_2 = '/content/output/json/Dristi_Padam/'

output_path_3 = '/content/output/json/Santosh_vriksh/'
output_path_4 = '/content/output/json/Sarthak_Vriksh/'

case1 = read_vec(output_path_1)
print(len(case1))
#print(at)

case2 = read_vec(output_path_2)
print(len(case2))

case3 = read_vec(output_path_3)
print(len(case3))

case4 = read_vec(output_path_4)
print(len(case4))

#alpha = 0.05
#alpha = 0.08
alpha = 0.1
#alpha = 0.02


print("simil_oks_1:", compute_oks(case1, case2, alpha))
print("simil_oks_2:", compute_oks(case3, case4, alpha))
print("simil_pdj_1:", compute_pdj(case1, case2, alpha))
print("simil_pdj_1:", compute_pdj(case3, case4, alpha))

print("dissimil_oks_1:", compute_oks(case1, case3, alpha))
print("dissimil_oks_2:", compute_oks(case2, case4, alpha))
print("dissimil_pdj_1:", compute_pdj(case1, case3, alpha))
print("dissimil_pdj_2:", compute_pdj(case2, case4, alpha))

2500
2500
2500
2500
simil_oks_1: 0.40759016548302096
simil_oks_2: 0.48734690328838176
simil_pdj_1: 0.3652
simil_pdj_1: 0.454
dissimil_oks_1: 0.4234780170517333
dissimil_oks_2: 0.31184113467882746
dissimil_pdj_1: 0.4216
dissimil_pdj_2: 0.2524
